# Zeca Afonso

In this notebook we train a voice conversion model for Zeca Afonso.

The notebook works in three parts:
1. Preparing the training dataset
2. Training the voice conversion model
3. Inference with the voice conversion model

## 1. Preparing the training dataset

### 1.1 Extract voice from Zeca Afonso's discography

The first step is to extract the voice from the songs. We will use the [`demucs`](https://github.com/facebookresearch/demucs) Hybrid Transformer model. 

In [ ]:
import subprocess
import tqdm
from pathlib import Path

# Iterate over the discography containing opus and mp3 files
for song in tqdm.tqdm(Path("dataset/zeca/discography").rglob("*.opus") | Path("dataset/zeca/discography").rglob("*.mp3")):
    command = f"demucs --two-stems=vocals {song} --out \"dataset/zeca/discography_vocals\" --filename \"{song.stem}.wav\""
    print(command)